# Instrument classification

Before starting model training, confirm whether you have GPUs available.

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


The following libraries can be loaded if installation instructions from the README file were completed.

In [2]:
import json
import librosa
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report

### Data loading and pre-processing

To start the analysis, we will load the OpenMic-2018 dataset, which should be present under `../data/raw/openmic-2018/`. The authors have provided a pre-processed dataset in the `openmic-2018.npz` file. This object contains contains VGGish features (X), True label probabilities (Y_true), binary masks (Y_masks) and sample keys (sample_key).

In [3]:
# Set the following parameter before running your analysis
DATA_ROOT = Path('../data/raw/openmic-2018/')

# Load openmic from npz object
OPENMIC = np.load(DATA_ROOT.joinpath('openmic-2018.npz'), allow_pickle=True)

# openmic contains VGGish features (X), True label probabilities (Y_true), binary masks (Y_masks) and sample keys
X, Y_true, Y_mask, sample_key = OPENMIC['X'], OPENMIC['Y_true'], OPENMIC['Y_mask'], OPENMIC['sample_key']

The OpenMic authors used VVGish features as inputs for their model. Here we will generate MFCCs following the method described by Blazke & Bostek in their publication (https://www.researchgate.net/publication/360046712_Musical_Instrument_Identification_Using_Deep_Learning_Approach). We overwrite the X object containing the VGGish features with our new MFCCs.

In [4]:
# Parameters for MFCC extraction, following Blazke & Bostek
n_fft = 1024  # Hamming window length (number of samples)
hop_length = 512  # Step size (number of samples)
n_mfcc = 40  # Number of MFCC bins

# Replace X by loading the .ogg files as MFCCs
mfcc_features = []

for i in tqdm(range(len(sample_key))):
    file = sample_key[i]
    dir = file[0:3]
    
    # Load audio file as waveform
    audio, sr = librosa.load(DATA_ROOT.joinpath('audio', dir, f"{file}.ogg"), sr=22050, mono=True)
    
    # Compute MFCCs
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    
    # Transpose to match time steps on the first axis
    mfcc_features.append(mfcc.T)

# Convert the list of arrays into a numpy array
X = np.array(mfcc_features)

print(f"Each input audio file now has the following shape: {X[0].shape}")

100%|██████████| 20000/20000 [09:47<00:00, 34.06it/s]


Each input audio file now has the following shape: (431, 40)


The OpenMic authors have a pre-defined balanced test-train split. We will use their method of loading the partition files.

In [5]:
# Load the class map and the train / test split
with open(DATA_ROOT.joinpath('class-map.json'), 'r') as f:
    class_map = json.load(f)

# Let's split the data into the training and test set
split_train = pd.read_csv(DATA_ROOT.joinpath('partitions/split01_train.csv'), 
                          header=None, squeeze=True)
split_test = pd.read_csv(DATA_ROOT.joinpath('partitions/split01_test.csv'), 
                         header=None, squeeze=True)

train_set = set(split_train)
test_set = set(split_test)


# We now identify the IDs that are part of the train and test set
idx_train, idx_test = [], []

for idx, n in enumerate(sample_key):
    if n in train_set:
        idx_train.append(idx)
    elif n in test_set:
        idx_test.append(idx)
    else:
        # This should never happen, but better safe than sorry.
        raise RuntimeError('Unknown sample key={}! Abort!'.format(sample_key[n]))
        
# Finally, cast the idx_* arrays to numpy structures
idx_train = np.asarray(idx_train)
idx_test = np.asarray(idx_test)

# Finally, we use the split indices to partition the features, labels, and masks
X_train = X[idx_train]
X_test = X[idx_test]

Y_true_train = Y_true[idx_train]
Y_true_test = Y_true[idx_test]

Y_mask_train = Y_mask[idx_train]
Y_mask_test = Y_mask[idx_test]

# Prepare binary labels for classification
Y_train_binary = (Y_true_train > 0.5).astype(int)
Y_test_binary = (Y_true_test > 0.5).astype(int)

/var/folders/lm/6s5grh814c71c5dc8z3w54zr0000gq/T/ipykernel_91334/280367173.py:6: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  split_train = pd.read_csv(DATA_ROOT.joinpath('partitions/split01_train.csv'),
/var/folders/lm/6s5grh814c71c5dc8z3w54zr0000gq/T/ipykernel_91334/280367173.py:8: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  split_test = pd.read_csv(DATA_ROOT.joinpath('partitions/split01_test.csv'),


### Model training

We will train one model, and generate an evaluation report, for each of the 20 instruments present in the OpenMi-2018 dataset. The model architecture is based on the one used by Blazke & Bostek, with some modifications.

In [7]:
models = {}
reports = {}

for instrument in class_map:

    print(instrument)

    # Map the instrument name to its column number
    inst_num = class_map[instrument]
        
    # Extract the instrument data
    train_inst = Y_mask_train[:, inst_num]
    test_inst = Y_mask_test[:, inst_num]
    
    X_train_inst = X_train[train_inst]
    Y_true_train_inst = Y_true_train[train_inst, inst_num] >= 0.5

    X_test_inst = X_test[test_inst]
    Y_true_test_inst = Y_true_test[test_inst, inst_num] >= 0.5

    # Set the input shape based on MFCC output dimensions
    input_shape = (431, 40, 1)

    # Define the model
    model = Sequential()
    model.add(Input(shape=input_shape))

    # First Conv2D layer with 128 filters
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3)) 

    # Second Conv2D layer with 64 filters
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3)) 

    # Third Conv2D layer with 32 filters
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3)) 

    # Flatten the output from convolutional layers
    model.add(Flatten())

    # Dense layers
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3)) 
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))  
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with custom metrics
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Reduce learning rate on plateau
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6, verbose=1)

    # Fit the model
    model.fit(X_train_inst, Y_true_train_inst, epochs=25, batch_size=32, validation_split=0.2, callbacks=[reduce_lr])


    # Generate and print a training report per instrument during training
    print(f'Instrument: {instrument}')
    Y_pred_test = model.predict(X_test_inst) > 0.5  # Convert predictions to binary
    report = classification_report(Y_true_test_inst, Y_pred_test)
    print(report)

    # Store the classifier and report in their dictionaries
    models[instrument] = model
    reports[instrument] = report

accordion
Epoch 1/25
39/39 [==============================] - 5s 95ms/step - loss: 3.8720 - accuracy: 0.6550 - val_loss: 1.5790 - val_accuracy: 0.7785 - lr: 0.0010
Epoch 2/25
39/39 [==============================] - 3s 69ms/step - loss: 2.8198 - accuracy: 0.6656 - val_loss: 0.6782 - val_accuracy: 0.7134 - lr: 0.0010
Epoch 3/25
39/39 [==============================] - 3s 70ms/step - loss: 4.1519 - accuracy: 0.6574 - val_loss: 1.1774 - val_accuracy: 0.7557 - lr: 0.0010
Epoch 4/25
39/39 [==============================] - 3s 72ms/step - loss: 5.1101 - accuracy: 0.6688 - val_loss: 2.8313 - val_accuracy: 0.7785 - lr: 0.0010
Epoch 5/25
38/39 [============================>.] - ETA: 0s - loss: 4.8781 - accuracy: 0.6900
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
39/39 [==============================] - 3s 69ms/step - loss: 4.8686 - accuracy: 0.6884 - val_loss: 3.9033 - val_accuracy: 0.7785 - lr: 0.0010
Epoch 6/25
39/39 [==============================] - 3s 69ms/

Epoch 1/25
44/44 [==============================] - 5s 96ms/step - loss: 12.7377 - accuracy: 0.5568 - val_loss: 0.9201 - val_accuracy: 0.6638 - lr: 0.0010
Epoch 2/25
44/44 [==============================] - 3s 70ms/step - loss: 9.3876 - accuracy: 0.5769 - val_loss: 1.7860 - val_accuracy: 0.6437 - lr: 0.0010
Epoch 3/25
44/44 [==============================] - 3s 72ms/step - loss: 10.2989 - accuracy: 0.5797 - val_loss: 4.9175 - val_accuracy: 0.4770 - lr: 0.0010
Epoch 4/25
43/44 [============================>.] - ETA: 0s - loss: 12.9973 - accuracy: 0.5865
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
44/44 [==============================] - 3s 69ms/step - loss: 12.8735 - accuracy: 0.5876 - val_loss: 2.4349 - val_accuracy: 0.6293 - lr: 0.0010
Epoch 5/25
44/44 [==============================] - 3s 71ms/step - loss: 10.1948 - accuracy: 0.6092 - val_loss: 2.1539 - val_accuracy: 0.6293 - lr: 1.0000e-04
Epoch 6/25
44/44 [==============================] - 3s 69ms/s

Epoch 1/25
36/36 [==============================] - 4s 99ms/step - loss: 6.3628 - accuracy: 0.6140 - val_loss: 7.6317 - val_accuracy: 0.7263 - lr: 0.0010
Epoch 2/25
36/36 [==============================] - 3s 74ms/step - loss: 7.5324 - accuracy: 0.6228 - val_loss: 2.6473 - val_accuracy: 0.6561 - lr: 0.0010
Epoch 3/25
36/36 [==============================] - 3s 71ms/step - loss: 3.5802 - accuracy: 0.6860 - val_loss: 1.8760 - val_accuracy: 0.5404 - lr: 0.0010
Epoch 4/25
36/36 [==============================] - 3s 72ms/step - loss: 3.4764 - accuracy: 0.6675 - val_loss: 3.9212 - val_accuracy: 0.7018 - lr: 0.0010
Epoch 5/25
36/36 [==============================] - 2s 69ms/step - loss: 5.7923 - accuracy: 0.7018 - val_loss: 4.9996 - val_accuracy: 0.5088 - lr: 0.0010
Epoch 6/25
36/36 [==============================] - ETA: 0s - loss: 6.1274 - accuracy: 0.6789
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
36/36 [==============================] - 2s 70ms/step - los

Epoch 1/25
37/37 [==============================] - 4s 94ms/step - loss: 18.3047 - accuracy: 0.5559 - val_loss: 2.8938 - val_accuracy: 0.5700 - lr: 0.0010
Epoch 2/25
37/37 [==============================] - 3s 72ms/step - loss: 10.6001 - accuracy: 0.6302 - val_loss: 3.3012 - val_accuracy: 0.5973 - lr: 0.0010
Epoch 3/25
37/37 [==============================] - 3s 69ms/step - loss: 4.7640 - accuracy: 0.6328 - val_loss: 3.9740 - val_accuracy: 0.5904 - lr: 0.0010
Epoch 4/25
36/37 [============================>.] - ETA: 0s - loss: 4.4945 - accuracy: 0.6233
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
37/37 [==============================] - 3s 68ms/step - loss: 4.4686 - accuracy: 0.6260 - val_loss: 7.2773 - val_accuracy: 0.5870 - lr: 0.0010
Epoch 5/25
37/37 [==============================] - 3s 68ms/step - loss: 3.5805 - accuracy: 0.6550 - val_loss: 6.5757 - val_accuracy: 0.5904 - lr: 1.0000e-04
Epoch 6/25
37/37 [==============================] - 3s 69ms/step

Epoch 1/25
44/44 [==============================] - 5s 93ms/step - loss: 13.5137 - accuracy: 0.6032 - val_loss: 23.5568 - val_accuracy: 0.8109 - lr: 0.0010
Epoch 2/25
44/44 [==============================] - 3s 71ms/step - loss: 11.7996 - accuracy: 0.6304 - val_loss: 5.4640 - val_accuracy: 0.8109 - lr: 0.0010
Epoch 3/25
44/44 [==============================] - 3s 69ms/step - loss: 9.0809 - accuracy: 0.6569 - val_loss: 9.4607 - val_accuracy: 0.8109 - lr: 0.0010
Epoch 4/25
44/44 [==============================] - 3s 69ms/step - loss: 5.2480 - accuracy: 0.6691 - val_loss: 4.9972 - val_accuracy: 0.8109 - lr: 0.0010
Epoch 5/25
44/44 [==============================] - 3s 70ms/step - loss: 4.9019 - accuracy: 0.6390 - val_loss: 3.3836 - val_accuracy: 0.8080 - lr: 0.0010
Epoch 6/25
44/44 [==============================] - 3s 69ms/step - loss: 4.7862 - accuracy: 0.6719 - val_loss: 2.4269 - val_accuracy: 0.8052 - lr: 0.0010
Epoch 7/25
44/44 [==============================] - 3s 69ms/step - loss: 

...

Epoch 1/25
51/51 [==============================] - 5s 84ms/step - loss: 4.6044 - accuracy: 0.5650 - val_loss: 7.0182 - val_accuracy: 0.7353 - lr: 0.0010
Epoch 2/25
51/51 [==============================] - 3s 68ms/step - loss: 5.8784 - accuracy: 0.5931 - val_loss: 4.9396 - val_accuracy: 0.7353 - lr: 0.0010
Epoch 3/25
51/51 [==============================] - 3s 67ms/step - loss: 5.8338 - accuracy: 0.5699 - val_loss: 5.1830 - val_accuracy: 0.7328 - lr: 0.0010
Epoch 4/25
51/51 [==============================] - 3s 67ms/step - loss: 6.6799 - accuracy: 0.5821 - val_loss: 3.1115 - val_accuracy: 0.7353 - lr: 0.0010
Epoch 5/25
51/51 [==============================] - 3s 66ms/step - loss: 8.1085 - accuracy: 0.5760 - val_loss: 2.3854 - val_accuracy: 0.6887 - lr: 0.0010
Epoch 6/25
51/51 [==============================] - 3s 67ms/step - loss: 8.9061 - accuracy: 0.5895 - val_loss: 3.7348 - val_accuracy: 0.7353 - lr: 0.0010
Epoch 7/25
51/51 [==============================] - 3s 67ms/step - loss: 10.

Epoch 1/25
54/54 [==============================] - 7s 90ms/step - loss: 8.9394 - accuracy: 0.5405 - val_loss: 1.8786 - val_accuracy: 0.6838 - lr: 0.0010
Epoch 2/25
54/54 [==============================] - 4s 69ms/step - loss: 5.9678 - accuracy: 0.5563 - val_loss: 3.1975 - val_accuracy: 0.6581 - lr: 0.0010
Epoch 3/25
54/54 [==============================] - 4s 71ms/step - loss: 4.3680 - accuracy: 0.5622 - val_loss: 3.0266 - val_accuracy: 0.6511 - lr: 0.0010
Epoch 4/25
53/54 [============================>.] - ETA: 0s - loss: 3.5135 - accuracy: 0.5855
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
54/54 [==============================] - 4s 68ms/step - loss: 3.5350 - accuracy: 0.5839 - val_loss: 3.8983 - val_accuracy: 0.6534 - lr: 0.0010
Epoch 5/25
54/54 [==============================] - 4s 69ms/step - loss: 3.8495 - accuracy: 0.5687 - val_loss: 2.8622 - val_accuracy: 0.6557 - lr: 1.0000e-04
Epoch 6/25
54/54 [==============================] - 4s 71ms/step -

Epoch 1/25
46/46 [==============================] - 6s 108ms/step - loss: 4.0726 - accuracy: 0.6110 - val_loss: 1.6762 - val_accuracy: 0.6785 - lr: 0.0010
Epoch 2/25
46/46 [==============================] - 3s 73ms/step - loss: 3.3234 - accuracy: 0.5926 - val_loss: 1.3744 - val_accuracy: 0.6839 - lr: 0.0010
Epoch 3/25
46/46 [==============================] - 3s 71ms/step - loss: 4.1352 - accuracy: 0.6178 - val_loss: 5.6298 - val_accuracy: 0.3134 - lr: 0.0010
Epoch 4/25
46/46 [==============================] - 3s 70ms/step - loss: 3.5423 - accuracy: 0.6233 - val_loss: 1.6770 - val_accuracy: 0.6104 - lr: 0.0010
Epoch 5/25
46/46 [==============================] - ETA: 0s - loss: 3.5092 - accuracy: 0.6287
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
46/46 [==============================] - 3s 69ms/step - loss: 3.5092 - accuracy: 0.6287 - val_loss: 1.6236 - val_accuracy: 0.6294 - lr: 0.0010
Epoch 6/25
46/46 [==============================] - 3s 71ms/step - lo

Epoch 1/25
36/36 [==============================] - 5s 108ms/step - loss: 4.1813 - accuracy: 0.6093 - val_loss: 6.8000 - val_accuracy: 0.5267 - lr: 0.0010
Epoch 2/25
36/36 [==============================] - 3s 70ms/step - loss: 3.7718 - accuracy: 0.6280 - val_loss: 0.8840 - val_accuracy: 0.6619 - lr: 0.0010
Epoch 3/25
36/36 [==============================] - 2s 69ms/step - loss: 2.9963 - accuracy: 0.6762 - val_loss: 1.7361 - val_accuracy: 0.5480 - lr: 0.0010
Epoch 4/25
36/36 [==============================] - 3s 70ms/step - loss: 3.9989 - accuracy: 0.6566 - val_loss: 3.9773 - val_accuracy: 0.6868 - lr: 0.0010
Epoch 5/25
35/36 [============================>.] - ETA: 0s - loss: 5.1546 - accuracy: 0.6848
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
36/36 [==============================] - 2s 69ms/step - loss: 5.1500 - accuracy: 0.6851 - val_loss: 3.2070 - val_accuracy: 0.6477 - lr: 0.0010
Epoch 6/25
36/36 [==============================] - 3s 77ms/step - lo

Epoch 1/25
30/30 [==============================] - 5s 124ms/step - loss: 4.9687 - accuracy: 0.6050 - val_loss: 33.6732 - val_accuracy: 0.3824 - lr: 0.0010
Epoch 2/25
30/30 [==============================] - 2s 76ms/step - loss: 2.8626 - accuracy: 0.7374 - val_loss: 10.5836 - val_accuracy: 0.3824 - lr: 0.0010
Epoch 3/25
30/30 [==============================] - 2s 70ms/step - loss: 1.5949 - accuracy: 0.7910 - val_loss: 3.5070 - val_accuracy: 0.5630 - lr: 0.0010
Epoch 4/25
30/30 [==============================] - 2s 70ms/step - loss: 1.3690 - accuracy: 0.7605 - val_loss: 1.0491 - val_accuracy: 0.8277 - lr: 0.0010
Epoch 5/25
30/30 [==============================] - 2s 70ms/step - loss: 1.3248 - accuracy: 0.7815 - val_loss: 2.0012 - val_accuracy: 0.6933 - lr: 0.0010
Epoch 6/25
30/30 [==============================] - 2s 71ms/step - loss: 1.7748 - accuracy: 0.7721 - val_loss: 2.3795 - val_accuracy: 0.7521 - lr: 0.0010
Epoch 7/25
30/30 [==============================] - ETA: 0s - loss: 3.174

### Saving models and reports to disk
Saving the result, for potential future re-use.

In [9]:
# Directory to save models and reports
save_dir = Path('../models')

# Iterate over each instrument and save the model and report
for instrument in models:
    # Construct the file paths
    model_path = save_dir.joinpath(f'{instrument}_model.h5')
    report_path = save_dir.joinpath(f'{instrument}_report.csv')

    # Save the Keras model
    models[instrument].save(model_path)
    print(f'Model saved to {model_path}')

    # Save the classification report to a text file
    with open(report_path, 'w') as file:
        file.write(f'Classification Report for {instrument}:\n')
        file.write(reports[instrument])
    print(f'Report saved to {report_path}')

/Users/molendijk.jeffrey/.pyenv/versions/3.11.7/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to ../models/accordion_model.h5
Report saved to ../models/accordion_report.csv
Model saved to ../models/banjo_model.h5
Report saved to ../models/banjo_report.csv
Model saved to ../models/bass_model.h5
Report saved to ../models/bass_report.csv
Model saved to ../models/cello_model.h5
Report saved to ../models/cello_report.csv
Model saved to ../models/clarinet_model.h5
Report saved to ../models/clarinet_report.csv
Model saved to ../models/cymbals_model.h5
Report saved to ../models/cymbals_report.csv
Model saved to ../models/drums_model.h5
Report saved to ../models/drums_report.csv
Model saved to ../models/flute_model.h5
Report saved to ../models/flute_report.csv
Model saved to ../models/guitar_model.h5
Report saved to ../models/guitar_report.csv
Model saved to ../models/mallet_percussion_model.h5
Report saved to ../models/mallet_percussion_report.csv
Model saved to ../models/mandolin_model.h5
Report saved to ../models/mandolin_report.csv
Model saved to ../models/organ_model.h5